#### **1. 导入模块**

导入 Python 标准库和本项目自定义库

In [9]:
# 标准库
import os
import sys

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
from src.utils import load_corpus, save_results
from src.annotator.mt_generation import MTGenerator, translate_data

#### **2. 读取语料**

读取 TSV 格式的平行语料库

In [10]:
# 语料库目录 data/raw 包含两个 TSV 文件：
# test_data_001.tsv 和 test_data_002.tsv

# TSV 文件格式：
# 第一列：汉语原文，选自《鹿鼎记》第一回和第十回
# 第二列：英语译文，选自《The Deer and The Cauldron》（闵福德译）
data_dir = '../data/raw'
print(f"从 {data_dir} 目录读取数据 ...")

# 读取数据
data = load_corpus(data_dir)
print(f"成功读取 {len(data)} 条平行句对")

# 预览数据
# 导入后的数据以 DataFrame 格式存储
# 第一列：原文；第二列：译文；第三列：文件名
# 原文为 None，表示增译
# 译文为 None，表示减译
print("数据前 5 行如下：")
data.head()

从 ../data/raw 目录读取数据 ...
成功读取 300 条平行句对
数据前 5 行如下：


,source,target
1,江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。,Along a coastal road somewhere south of the Ya...
2,前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。,In each of the first three carts a single male...
3,后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。,"The four rear carts were occupied by women, th..."
4,女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。,The little girl was crying in a continuous wai...
5,囚车旁一清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！,"One of the soldiers marching alongside, irrita..."


#### **3. 加载模型**

加载大模型机器翻译模型

In [11]:
# 指定用于机器翻译的大模型
# 候选模型：
# qwen-flash, qwen-plus, qwen3-max, glm-4.7, deepseek-v3.2

# model_names 列表可选取多个模型
# 生成包含多版本译文的平行语料库
model_names = ['deepseek-v3.2', 'qwen3-max']

# 指定原文和译文语种
# 用于构建翻译任务提示词
# 提示词模版详见：
# src/prompt/mt_generation_prompt.py
src_lang = 'Chinese'
tgt_lang = 'English'
print("加载翻译模型 ...")
print(f"翻译方向：{src_lang} -> {tgt_lang}")

# 加载模型前，请登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账号
# 并在 llm_corpus_annotation/config 文件中设置 LLM_API_KEY=sk-********
models = {}
for model_name in model_names:
    models[model_name] = MTGenerator(
        model=model_name,
        src_lang=src_lang,
        tgt_lang=tgt_lang,
    )
loaded_models = list(models.keys())
print(f"模型加载完成！使用以下 {len(loaded_models)} 个模型生成译文：")
print(f"{('\n').join(loaded_models)}")

加载翻译模型 ...
翻译方向：Chinese -> English
模型加载完成！使用以下 2 个模型生成译文：
deepseek-v3.2
qwen3-max


#### **4. 批量翻译**

使用 DeepSeek-V3.2 和 Qwen3-Max 大模型，自动生成机器译文

构建包含人类和机器译文的多版本平行语料库

In [12]:
# 调用大模型 API 批量翻译原文

# 注意：
# 为节省 API 调用成本，本项目将大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果
# 因此，程序运行时间显示为 0.0 秒

# 首次调用 API 翻译 300 句原文
# 生成 2 个大模型译本（deepseek-v3.2 + qwen3-max）
# 大约需要 20 分钟

# 开始翻译
results = translate_data(data, models)

Translating: 100%|█████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 7822.08it/s]


#### **5. 保存结果**

将大模型译文保存于 JSON 文件

In [13]:
# 指定输出文件
out_file = f"../data/output/1_mt_generation.jsonl"
    
# 保存大模型译文
save_results(results, out_file)
print(f"结果已保存至 {out_file}\n")

结果已保存至 ../data/output/1_mt_generation.jsonl



#### **6. 查看结果**

预览大模型译文，查看人类-机器译文异同

In [14]:
# 预览前 3 条数据
# 输出人类译文、Deepseek译文和 Qwen3-Max 译文
for res in results[:3]:
    print(f"\n{'[ID]':<15}{res['id']}")
    print(f"{'[Source]':<15}{res['source']['raw_text']}")
    print("-" * 60)
    print(f"{'[Human]':<15}{res['targets']['human']['raw_text']}")
    print(f"{'[DeepSeek]':<15}{res['targets']['deepseek-v3.2']['raw_text']}")
    print(f"{'[Qwen]':<15}{res['targets']['qwen3-max']['raw_text']}")
    print("=" * 60)


[ID]           000001
[Source]       江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
------------------------------------------------------------
[Human]        Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.
[DeepSeek]     Along a broad road near the coast south of the Yangtze River, a troop of Qing soldiers, armed with swords and spears, escorted seven prison carts northward, braving the biting wind and cold.
[Qwen]         On a major road near the coast in Jiangnan, a squad of Qing soldiers, armed with swords and spears, escorted seven prison carts northward through biting wind and bitter cold.

[ID]           000002
[Source]       前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。
------------------------------------------------------------
[Human]        In each of the first three carts a single male prisoner was caged, identifiable b